In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating new project at `c:\Users\nikla\git\AMOProject`
  No Changes to `C:\Users\nikla\git\AMOProject\Project.toml`
  No Changes to `C:\Users\nikla\git\AMOProject\Manifest.toml`


In [23]:
using JuMP, Distributions, HiGHS, Clustering, Distances, Plots, StatsPlots, Statistics, PlotlyJS, StatsBase, Ipopt

In [13]:
# Generate 288 distribution vectors
function generate_distributions(si::Float64, consumption_mean::Float64, temp_mean::Float64, hour::Int)
    if hour < 6 || hour > 22
        solar = Beta(1, 1000)  # Close to zero but keeps Beta distribution structure
        temperature = Normal(temp_mean-4, 9)
        consumption = Normal(consumption_mean/(consumption_mean-1), consumption_mean / 11)    
    else
        β = ((si * (1 + si) / (si / 10)) - 1) * (1 - si)
        α = si * β / (1 - si)
        solar = Beta(α, β)
        temperature = Normal(temp_mean, 9)
        consumption = Normal(consumption_mean, consumption_mean / 11)    

    end
    return [solar, consumption, temperature]
end

dist_vec = Vector{Vector{Distribution}}(undef, 12 * 24)

for month in 1:12
    for hour in 1:24
        # Assume some seasonal variations
        si = 0.25 + 0.1 * sin(2π * month / 12)  # Varying solar irradiance
        si = hour < 6 || hour > 22 ? 0.0 : si   # Zero irradiance at night
        consumption_mean = 90.0 + 10 * sin(2π * hour / 24)  # Daily load cycle
        temp_mean = 13.0 + 5 * cos(2π * month / 12)  # Seasonal temperature
        
        # Store the distribution vector
        dist_vec[(month - 1) * 24 + hour] = generate_distributions(si, consumption_mean, temp_mean, hour)
    end
end

# Check an example
dist_vec[1]  # First hour of January

3-element Vector{Distribution}:
 Beta{Float64}(α=1.0, β=1000.0)
 Normal{Float64}(μ=1.0109184382295962, σ=8.417108222820474)
 Normal{Float64}(μ=13.330127018922195, σ=9.0)

In [21]:
# Function to sample, cluster, and compute probabilities
function cluster_scenarios_with_probabilities(dist_vec, num_samples=5000, num_clusters=5)
    scenarios = []
    probabilities = []
    
    for (i, dist_set) in enumerate(dist_vec)
        # Generate 5000 samples for each variable
        samples = hcat(rand(dist_set[1], num_samples),  # Solar Irradiance
                       rand(dist_set[2], num_samples),  # Energy Consumption
                       rand(dist_set[3], num_samples)) # Temperature
        
        # Ensure non-negativity
        samples[:, 1] .= max.(samples[:, 1], 0)  # Solar Irradiance ≥ 0
        samples[:, 2] .= max.(samples[:, 2], 0)  # Consumption ≥ 0

        # Apply k-means clustering
        result = kmeans(samples', num_clusters)  # Clustering (transpose for correct shape)
        centroids = result.centers'  # Get cluster centers (transpose back)
        
        # Compute probabilities (relative frequency of points in each cluster)
        cluster_counts = countmap(result.assignments)  # Count points per cluster
        probs = [cluster_counts[j] / num_samples for j in 1:num_clusters]  # Normalize

        # Store results
        push!(scenarios, centroids)
        push!(probabilities, probs)
    end
    
    return scenarios, probabilities
end

# Generate 5 scenarios and their probabilities for all 288 time periods
scenarios_5, scenario_probs = cluster_scenarios_with_probabilities(dist_vec)

# Check an example
println("Scenarios for January, 1st hour: ", scenarios_5[1]) 
println("Probabilities for January, 1st hour: ", scenario_probs[1], " sum ", sum(scenario_probs[1]))  

Scenarios for January, 1st hour: [0.0010280930697278133 1.6523667043996533 0.9890543072374182; 0.0009951774445681 11.219511481003131 20.018763702778173; 0.0009478371166922513 1.5519123836840234 24.74745862624762; 0.0010057383473341588 1.3548433977594132 12.745069837658939; 0.0010046247756005603 12.212646737195456 6.921588650536541]
Probabilities for January, 1st hour: [0.193, 0.1212, 0.2004, 0.3686, 0.1168] sum 1.0


## Optimization Model

### Scalars

In [29]:
T = 288  # 12 months * 24 hours
num_scenarios = 5  # 5 clusters per time step

SCC = 50  # Social cost of carbon ($/ton CO2)
E_cap = 1000  # Emissions cap (optional)
C_inv_per_kW = 1000  # Investment cost per kW (per year)

c_grid_buy = 0.25  # Cost of buying from grid ($/kWh)
c_grid_feed = 0.10  # Revenue from feeding into the grid ($/kWh)
e_grid = 0.5  # Emission intensity of grid (tCO2/kWh)

# PV Parameters
η_0 = 0.18  # Nominal PV efficiency at 25°C
β = 0.004  # Temperature coefficient (per °C)
T_ref = 25  # Reference temperature (°C)
A = 1  # Area of PV system (arbitrary unit, related to y_PV)

# === PV Efficiency as a function of Temperature === #
    function efficiency(T_t)
        return η_0 * (1 - β * (T_t - T_ref))  # Efficiency model
    end

# === Load Scenario Data === #

# Solar irradiance: column 1
S_t_omega = [[scenarios_5[t][ω, 1] for ω in 1:num_scenarios] for t in 1:T]

# Consumption: column 2
L_t_omega = [[scenarios_5[t][ω, 2] for ω in 1:num_scenarios] for t in 1:T]

# Temperature: column 3
T_t_omega = [[scenarios_5[t][ω, 3] for ω in 1:num_scenarios] for t in 1:T]

# Scenario probabilities
Φ_t_omega = [[scenario_probs[t][ω] for ω in 1:num_scenarios] for t in 1:T] 

288-element Vector{Vector{Float64}}:
 [0.193, 0.1212, 0.2004, 0.3686, 0.1168]
 [0.125, 0.1094, 0.195, 0.3482, 0.2224]
 [0.1158, 0.2184, 0.3606, 0.1154, 0.1898]
 [0.1604, 0.1514, 0.1418, 0.2672, 0.2792]
 [0.2136, 0.1982, 0.1186, 0.3658, 0.1038]
 [0.2476, 0.1586, 0.2282, 0.209, 0.1566]
 [0.2524, 0.1816, 0.1972, 0.1862, 0.1826]
 [0.2518, 0.156, 0.2124, 0.2304, 0.1494]
 [0.262, 0.1566, 0.1532, 0.2274, 0.2008]
 [0.201, 0.2322, 0.2818, 0.134, 0.151]
 ⋮
 [0.1758, 0.1914, 0.1802, 0.1888, 0.2638]
 [0.1536, 0.1558, 0.2036, 0.2676, 0.2194]
 [0.2662, 0.1776, 0.132, 0.2104, 0.2138]
 [0.2158, 0.2886, 0.137, 0.217, 0.1416]
 [0.1816, 0.1646, 0.1482, 0.2436, 0.262]
 [0.1452, 0.1658, 0.208, 0.2568, 0.2242]
 [0.203, 0.1854, 0.2578, 0.1686, 0.1852]
 [0.111, 0.3578, 0.1886, 0.1148, 0.2278]
 [0.278, 0.1474, 0.2764, 0.1326, 0.1656]

### Model

#### Upper Level

In [ ]:
model = Model(Ipopt.Optimizer)

@variable(model, y_PV >= 0)  # PV capacity to be decided
@variable(model, P_PV[1:T, 1:num_scenarios] >= 0)  # PV power production for each time step and scenario

# PV production is calculated based on efficiency function
@constraint(model, [t=1:T, ω=1:num_scenarios],
    P_PV[t, ω] == efficiency(T_t_omega[t][ω]) * A * S_t_omega[t][ω]
)

# Emissions Calculation
@expression(model, E_op,
    sum(Φ_t_omega[t][ω] * e_grid * G_in[t, ω] for t in 1:T, ω in 1:num_scenarios)
)

# Upper Level Objective: Minimize Total Cost
@objective(model, Min,
    C_inv_per_kW * y_PV  # Investment Cost
    + C_op  # Expected Operating Cost
    + SCC * E_op  # Expected Emissions Cost
)

VariableNotOwned{VariableRef}: VariableNotOwned{VariableRef}(P_PV[1,1]): the variable P_PV[1,1] cannot be used in this model because
it belongs to a different model.

## Explanation

Each variable belongs to a particular model. You cannot use a variable
inside a model to which it does not belong. For example:

```julia
model = Model()
@variable(model, x >= 0)
model2 = Model()
@objective(model2, Min, x)  # Errors because x belongs to `model`
```

One common cause of this error is working with a main problem and a
subproblem.

```julia
model = Model()
@variable(model, x, Bin)
subproblem = Model()
@variable(subproblem, 0 <= x <= 1)
# The Julia binding `x` now refers to `subproblem[:x]`, not `model[:x]`
@objective(model, Min, x)  # Errors because x belongs to `subproblem`
# do instead
@objective(model, Min, model[:x])
```

#### Lower Level

In [26]:
# Lower Level Variables (Operational Dispatch)
@variable(model, G_in[t=1:T, ω=1:num_scenarios] >= 0)  # Grid import
@variable(model, G_out[t=1:T, ω=1:num_scenarios] >= 0)  # Grid feed-in
@variable(model, P_PV[t=1:T, ω=1:num_scenarios] >= 0)  # PV production
# Meeting the demand
@constraint(model, [t=1:T, ω=1:num_scenarios],
    L_t_omega[t][ω] == y_PV * S_t_omega[t][ω] + G_in[t, ω] - G_out[t, ω] - P_PV[t, ω]
)
# === Lower Level Objective: Minimize Operating Cost === #
@expression(model, C_op,
    sum(Φ_t_omega[t][ω] * (c_grid_buy * G_in[t, ω] - c_grid_feed * G_out[t, ω]) 
        for t in 1:T, ω in 1:num_scenarios)
)

UndefVarError: UndefVarError: `L_t_omega` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Solving

In [27]:
print(model)
print(model[:variables])
print(model[:constraints])

# === Solve Model === #
#optimize!(model)


Feasibility
Subject to
 y_PV >= 0
 G_in[1,1] >= 0
 G_in[2,1] >= 0
 G_in[3,1] >= 0
 G_in[4,1] >= 0
 G_in[5,1] >= 0
 G_in[6,1] >= 0
 G_in[7,1] >= 0
 G_in[8,1] >= 0
 G_in[9,1] >= 0
 G_in[10,1] >= 0
 G_in[11,1] >= 0
 G_in[12,1] >= 0
 G_in[13,1] >= 0
 G_in[14,1] >= 0
 G_in[15,1] >= 0
 G_in[16,1] >= 0
 G_in[17,1] >= 0
 G_in[18,1] >= 0
 G_in[19,1] >= 0
 G_in[20,1] >= 0
 G_in[21,1] >= 0
 G_in[22,1] >= 0
 G_in[23,1] >= 0
 G_in[24,1] >= 0
 G_in[25,1] >= 0
 G_in[26,1] >= 0
 G_in[27,1] >= 0
 G_in[28,1] >= 0
 G_in[29,1] >= 0
 G_in[30,1] >= 0
 G_in[31,1] >= 0
 G_in[32,1] >= 0
 G_in[33,1] >= 0
 G_in[34,1] >= 0
 G_in[35,1] >= 0
 G_in[36,1] >= 0
 G_in[37,1] >= 0
 G_in[38,1] >= 0
 G_in[39,1] >= 0
 G_in[40,1] >= 0
 G_in[41,1] >= 0
 G_in[42,1] >= 0
 G_in[43,1] >= 0
 G_in[44,1] >= 0
 G_in[45,1] >= 0
 G_in[46,1] >= 0
 G_in[47,1] >= 0
 G_in[48,1] >= 0
 G_in[49,1] >= 0
[[...4221 constraints skipped...]]
 P_PV[239,5] >= 0
 P_PV[240,5] >= 0
 P_PV[241,5] >= 0
 P_PV[242,5] >= 0
 P_PV[243,5] >= 0
 P_PV[244,5] >= 0

KeyError: KeyError: key :variables not found